In [1]:
import pandas as pd

In [8]:
study_datasheet = 'Studienergebnisse.xlsx'
config_datasheet = 'Vokabeltraining.xlsx'
config_pre_sosci_datasheet = r'./results/preprocessed.xlsx'
config_post_sosci_datasheet = r'./results/preprocessed_posttest.xlsx'

In [11]:
raw_study = pd.read_excel(study_datasheet)
raw_if = pd.read_excel(config_datasheet, sheet_name ="if-else")
raw_for = pd.read_excel(config_datasheet, sheet_name= "for")
raw_while = pd.read_excel(config_datasheet, sheet_name="while")
raw_do_while = pd.read_excel(config_datasheet, sheet_name="do-while")
raw_pre_sosci = pd.read_excel(config_pre_sosci_datasheet)
raw_post_sosci = pd.read_excel(config_post_sosci_datasheet)

df_if = pd.DataFrame(raw_if)
df_for = pd.DataFrame(raw_for)
df_while = pd.DataFrame(raw_while)
df_do_while = pd.DataFrame(raw_do_while)
df_pre_sosci = pd.DataFrame(raw_pre_sosci)
df_post_sosci = pd.DataFrame(raw_post_sosci)

df_study = pd.DataFrame(raw_study)

df_if = df_if.rename(columns={'Max Punkte': 'Max Punkte-if', 'Fehler' : 'Fehler-if', 'Zeit':'Zeit-if'})
df_for = df_for.rename(columns={'Max Punkte': 'Max Punkte-for', 'Fehler' : 'Fehler-for', 'Zeit':'Zeit-for'})
df_while = df_while.rename(columns={'Max Punkte': 'Max Punkte-while', 'Fehler' : 'Fehler-while', 'Zeit':'Zeit-while'})
df_do_while = df_do_while.rename(columns={'Max Punkte': 'Max Punkte-do-while', 'Fehler' : 'Fehler-do-while', 'Zeit':'Zeit-do-while'})


df_study = df_study.drop(columns=['Vokabeltraining-if', 'Vokabeltraining-for', 'Vokabeltraining-while', 'Vokabeltraining-do-while', 'Pre-Test/SoSci-Survey', 'Post-Test/SoSci-Survey'])

df_vocabulary = pd.merge(df_if, df_for, how='outer', on='Unique Code')
df_vocabulary = pd.merge(df_vocabulary, df_while, how='outer', on='Unique Code')
df_vocabulary = pd.merge(df_vocabulary, df_do_while, how='outer', on='Unique Code')


tmp_pre = pd.DataFrame(columns=[])
tmp_post = pd.DataFrame(columns=[])

codes_pre = df_pre_sosci["Unique Code"].unique()
codes_post = df_post_sosci["Unique Code"].unique()

counter = 0

unique_code_pre_list = []
point_pre_list = []
max_point_pre_list = []

for code in codes_pre:
    tmp = df_pre_sosci.loc[df_pre_sosci["Unique Code"]== code]["Correctness"]
    for t in tmp:
        if t == True:
            counter += 1
    unique_code_pre_list.append(code)
    point_pre_list.append(counter)
    max_point_pre_list.append(10)
    counter = 0

unique_code_post_list = []
point_post_list = []
max_point_post_list = []

for code in codes_post:
    tmp = df_post_sosci.loc[df_post_sosci["Unique Code"]== code]["Correctness"]
    for t in tmp:
        if t == True:
            counter += 1
    unique_code_post_list.append(code)
    point_post_list.append(counter)
    max_point_post_list.append(10)
    counter = 0

tmp_pre['Unique Code'] = unique_code_pre_list
tmp_pre['Punkte-Pretest-SoSci-Survey'] = point_pre_list
tmp_pre['Max Punkte-Pretest-SoSci-Survey'] = max_point_pre_list

tmp_post['Unique Code'] = unique_code_post_list
tmp_post['Punkte-Posttest-SoSci-Survey'] = point_post_list
tmp_post["Max Punkte-Posttest-SoSci-Survey"] = max_point_post_list
tmp_post


df_study = pd.merge(df_study, tmp_pre, how='outer', on='Unique Code')
df_study = pd.merge(df_study, tmp_post, how='outer', on='Unique Code')
df_study = pd.merge(df_study, df_vocabulary, how='outer', on='Unique Code')


df_study = df_study[['Unique Code', 'Punkte-Pretest-SoSci-Survey', 'Max Punkte-Pretest-SoSci-Survey', 'Pre-Test/Programmieraufgaben',
                     'Vokabeltraining-if', 'Max Punkte-if', 'Fehler-if', 'Zeit-if',
                     'Vokabeltraining-for', 'Max Punkte-for', 'Fehler-for', 'Zeit-for',
                     'Vokabeltraining-while', 'Max Punkte-while', 'Fehler-while', 'Zeit-while',
                     'Vokabeltraining-do-while', 'Max Punkte-do-while', 'Fehler-do-while', 'Zeit-do-while',
                     'Punkte-Posttest-SoSci-Survey', 'Max Punkte-Posttest-SoSci-Survey', 'Post-Test/Programmieraufgaben',
                     'd2', 'DESIGMA-A', 'C-Test', 'WST rezeptiv', 'WST produktiv']]



In [12]:
df_study = df_study.sort_values("Unique Code")

df_study.to_excel("results/Test.xlsx", index=False)

In [13]:
writer = pd.ExcelWriter("results/Test.xlsx") 
df_study.to_excel(writer, sheet_name='Seite 1', index=False, na_rep='NaN')

for column in df_study:
    column_length = max(df_study[column].astype(str).map(len).max(), len(column))
    col_idx = df_study.columns.get_loc(column)
    writer.sheets['Seite 1'].set_column(col_idx, col_idx, column_length)

writer.save()